This repo is based on DiffScaler, https://github.com/DSIP-FBK/DiffScaler/tree/main/src

You are already acquainted with Pytorch now as a fanstastic library for writing, modifying, testing and scaling up the code. In this notebook, let us learn a cool wrapper library for Pytorch known as Pytorch Lightning

A great resource to learn about the differences between Pytorch and Pytorch Lightning and how Pytorch Lightning makes your life easier : https://www.geeksforgeeks.org/deep-learning/pytorch-vs-pytorch-lightning/

Please note that according to this TA, you can`t really appreciate the resurcefulness of Pytorch lightning without learning the basics of Pytorch. Hence, take this tutorial as a 10,000 ft overview of what Pytorch and Pytorch lightning actually do. 

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset, random_split, TensorDataset
import torch.nn.functional as F
from lightning import LightningModule, LightningDataModule, Trainer
from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import numpy as np
from omegaconf import OmegaConf
import hydra
import random
import zstandard
import os
import glob
import io
import xarray as xr
import rasterio
import rioxarray
import json
import yaml

The original paper this notebook is based on, can be accessed at https://gmd.copernicus.org/articles/18/2051/2025/ . There are originally 14 input channels to predict 2 chanels ultimately : temperature and wind speed. For the purposes of this notebook (due to storage limits and memory issues), we shall only be using low res 2m temperature to predict the high res 2m temperature target field, using hourly data from a single year (2020). 

 But to supplement our endeavours in the form of giving more data to the model, we will also feed in static inputs (inputs that do not change or evolve over time) which are 1. digital elevation model (DEM), 2. land cover categories and 3. latitudinal bands. The functions to do so can be found in the preprocessing file, and we import them here for ease of analysis. 

In [2]:
from preprocessing import load_and_normalise, decompress_zst_pt, get_file_list, collate_fn

In [3]:
# @title
class DownscalingDataset(Dataset):
    def __init__(self, file_list, static_vars, low_2mt_mean, low_2mt_std):
        self.file_list = file_list  # List of (high_file, low_file, date)
        self.static_vars = static_vars  # for a list of static variables, refer the preprocessing file which loads and normalises these datasets
        self.low_2mt_mean = low_2mt_mean
        self.low_2mt_std = low_2mt_std
        self.samples = []
        # Each file contains 24 hourly samples
        for hf, lf, date in self.file_list:
            for hour in range(24):
                self.samples.append((hf, lf, hour))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        hf, lf, hour = self.samples[idx]
        # Loading here high-res and low-res data for the given hour
        high_data = decompress_zst_pt(hf)
        low_data = decompress_zst_pt(lf)
        # the high res and low res (upsampled) low res files are being loaded here
        high_t2m = high_data[hour]["2mT"].float()  #[672, 576]
        low_t2m = low_data[hour]["2mT"].float()    #[84, 72]


        high_t2m = high_t2m.unsqueeze(0)  #[1, 672, 576]
        low_t2m = low_t2m.unsqueeze(0)    #[1, 84, 72]
        # Static vars
        dem = self.static_vars["dem"].unsqueeze(0)  #[1, 672, 576]
        lat = self.static_vars["lat"].unsqueeze(0)  #[1, 672, 576]
        lc = self.static_vars["lc"]                 #[bands, 672, 576] ,,,by bands we mean land cover bands here

        #Finally, this returns a dictionary with the following keys and values (variables)
        return {
            "low_2mT": low_t2m,        #[1, 84, 72]
            "high_2mT": high_t2m,      #[1, 672, 576]
            "dem": dem,                #[1, 672, 576]
            "lat": lat,                #[1, 672, 576]
            "lc": lc                   #[bands, 672, 576]
        }

In [4]:
# @title
class DownscalingDataModule(LightningDataModule):
    def __init__(self, batch_size, val_frac, test_frac, num_workers, static_dir, save_stats_json):
        super().__init__()
        self.batch_size = batch_size
        self.val_frac = val_frac
        self.test_frac = test_frac
        self.num_workers = num_workers
        self.static_dir = static_dir
        self.save_stats_json = save_stats_json

    def setup(self, stage=None):  
        static_vars, stats = load_and_normalise(
            self.static_dir,
            val_frac=self.val_frac,
            test_frac=self.test_frac,
            save_stats_json=self.save_stats_json
        )
        file_list = get_file_list()
        dataset = DownscalingDataset(
            file_list, static_vars,
            low_2mt_mean=stats["low_2mt_mean"],
            low_2mt_std=stats["low_2mt_std"]
        )
        N = len(dataset)
        n_val = int(self.val_frac * N)
        n_test = int(self.test_frac * N)
        n_train = N - n_val - n_test

        # Contiguous splits (no shuffling)
        self.train_set = torch.utils.data.Subset(dataset, range(0, n_train))
        self.val_set = torch.utils.data.Subset(dataset, range(n_train, n_train + n_val))
        self.test_set = torch.utils.data.Subset(dataset, range(n_train + n_val, N))


    def train_dataloader(self):
        return DataLoader(self.train_set, batch_size=self.batch_size, shuffle=True, num_workers=self.num_workers, collate_fn=collate_fn)

    def val_dataloader(self):
        return DataLoader(self.val_set, batch_size=self.batch_size, shuffle=False, num_workers=self.num_workers, collate_fn=collate_fn)

    def test_dataloader(self):
        return DataLoader(self.test_set, batch_size=self.batch_size, shuffle=False, num_workers=self.num_workers, collate_fn=collate_fn)

    @property
    def test_dataset(self):
        return self.test_set.dataset



In [5]:
# @title
class UNet(nn.Module):
    def __init__(self, in_channels, out_channels, channels, kernel_size):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, channels[0], kernel_size, padding=1)
        self.conv2 = nn.Conv2d(channels[0], channels[1], kernel_size, padding=1)
        self.final = nn.Conv2d(channels[1], out_channels, 1)
    def forward(self, x):
        h = F.relu(self.conv1(x))
        h = F.relu(self.conv2(h))
        output = self.final(h)
        return output



In [6]:
class ResidualsDataModule(LightningDataModule):
    def __init__(self, unet_preds, residuals, batch_size):
        super().__init__()
        self.unet_preds = unet_preds  #Needed for conditonal generation
        self.residuals = residuals
        self.batch_size = batch_size

    def setup(self, stage=None):
        self.dataset = TensorDataset(self.unet_preds, self.residuals)

    def train_dataloader(self):
        return DataLoader(self.dataset, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.dataset, batch_size=self.batch_size, shuffle=False)

    def test_dataloader(self):
        return DataLoader(self.dataset, batch_size=self.batch_size, shuffle=False)

In [7]:
def kl_divergence(mean, log_var):
    kl = 0.5 * (log_var.exp() + mean.square() - 1.0 - log_var)
    return kl.mean()

def standard_normal_sampling(mean, log_var, num=None):
    std = log_var.mul(0.5).exp()
    shape = mean.shape
    if num is not None:
        shape = shape[:1] + (num,) + shape[1:]
        mean = mean[:, None, ...]
        std = std[:, None, ...]
    return mean + std * torch.randn(shape, device=mean.device)

In [8]:
class UNetLitModule(LightningModule):
    def __init__(
        self,
        net: torch.nn.Module = None,
        lr: float = 1e-3,
        optimizer: dict = None,
        scheduler: dict = None,
        loss_fn: torch.nn.Module = None,
        in_channels: int = 19,
        out_channels: int = 1,
        channels: list = [32, 16],
        kernel_size: int = 3,
        ckpt_path: str = None,
        ignore_keys: list = []
    ):
        super().__init__()
        self.save_hyperparameters(logger=False, ignore=['net', 'loss_fn'])
        self.net = net if net is not None else UNet(
            in_channels=in_channels,
            out_channels=out_channels,
            channels=channels,
            kernel_size=kernel_size
        )
        self.loss_fn = loss_fn if loss_fn is not None else nn.MSELoss()
        self.lr = lr
        self.hparams.optimizer = optimizer
        self.hparams.scheduler = scheduler

    def forward(self, x):
        return self.net(x)

    def model_step(self, batch):
        fuzzy_input, sharp_target = batch
        pred = self.forward(fuzzy_input)
        loss = self.loss_fn(pred, sharp_target)
        return loss, pred

    def training_step(self, batch, batch_idx):
        loss, pred = self.model_step(batch)
        self.log("train/loss", loss, on_step=True, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        loss, pred = self.model_step(batch)
        self.log("val/loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx):
        loss, pred = self.model_step(batch)
        self.log("test/loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        return loss

    def configure_optimizers(self):
        opt_cfg = self.hparams.get("optimizer", None)
        sch_cfg = self.hparams.get("scheduler", None)

        if opt_cfg and opt_cfg["type"] == "AdamW":
            optimizer = torch.optim.AdamW(
                self.parameters(),
                lr=opt_cfg.get("lr", self.lr),
                betas=tuple(opt_cfg.get("betas", (0.5, 0.9))),
                weight_decay=opt_cfg.get("weight_decay", 1e-3)
            )
        else:
            optimizer = torch.optim.Adam(
                self.parameters(),
                lr=opt_cfg.get("lr", self.lr) if opt_cfg else self.lr,
                weight_decay=opt_cfg.get("weight_decay", 1e-4) if opt_cfg else 1e-4
            )

        if sch_cfg and sch_cfg["type"] == "ReduceLROnPlateau":
            scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
                optimizer,
                patience=sch_cfg.get("patience", 2),
                factor=sch_cfg.get("factor", 0.25)
            )
            return {
                "optimizer": optimizer,
                "lr_scheduler": {
                    "scheduler": scheduler,
                    "monitor": sch_cfg.get("monitor", "val/loss"),
                    "frequency": 1,
                },
            }
        else:
            return optimizer

In [9]:
class VAE(nn.Module):
    def __init__(self, latent_dim=32, input_channels=2, input_height=672, input_width=576, encoder_channels=[8], decoder_channels=[8]):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(input_channels, encoder_channels[0], 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2), nn.Flatten()
        )
        
        encoded_size = encoder_channels[0] * (input_height // 2) * (input_width // 2)
        self.fc_mu = nn.Linear(encoded_size, latent_dim)
        self.fc_logvar = nn.Linear(encoded_size, latent_dim)
        self.fc_decode = nn.Linear(latent_dim, encoded_size)

        self.decoder = nn.Sequential(
            nn.Unflatten(1, (decoder_channels[0], input_height // 2, input_width // 2)),
            nn.ConvTranspose2d(decoder_channels[0] + 1, 1, 2, stride=2),  # +1 for unet_pred
            nn.Sigmoid()
        )

    def encode(self, x):
        h = self.encoder(x)
        return self.fc_mu(h), self.fc_logvar(h)

    def reparameterize(self, mu, logvar):
        std = (0.5 * logvar).exp()
        return mu + std * torch.randn_like(std)

    def decode(self, z, unet_pred):
        h = self.fc_decode(z)
        h = h.view(z.size(0), -1, unet_pred.shape[-2] // 2, unet_pred.shape[-1] // 2)
        # Downsample unet_pred to match h spatial size
        unet_pred_ds = F.interpolate(unet_pred, size=h.shape[-2:], mode='bilinear', align_corners=False)
        # Concatenate along channel dimension
        h_cat = torch.cat([h, unet_pred_ds], dim=1)
        return self.decoder(h_cat)

    def forward(self, x, unet_pred, sample_posterior=True):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar) if sample_posterior else mu
        recon = self.decode(z, unet_pred)
        return recon, mu, logvar

In [10]:
class VAELitModule(LightningModule):
    def __init__(self, latent_dim=32, lr=1e-3, kl_weight=0.001, input_channels=2, input_height=672, input_width=576, encoder_channels=[8], decoder_channels=[8], optimizer=None, scheduler=None, unet_module=None):
        super().__init__()
        self.save_hyperparameters(logger=False, ignore=['unet_module'])
        self.model = VAE(
            latent_dim=latent_dim,
            input_channels=input_channels,
            input_height=input_height,
            input_width=input_width,
            encoder_channels=encoder_channels,
            decoder_channels=decoder_channels,
        )
        self.lr = lr
        self.kl_weight = kl_weight
        self.hparams.optimizer = optimizer
        self.hparams.scheduler = scheduler
        self.unet = unet_module   #Note that this is required have conditional generation!

    def forward(self, x, sample_posterior=True):
        
        return self.model(x, sample_posterior)
    
    def _loss(self, batch):
        fuzzy_input, sharp_target = batch

        with torch.no_grad():
            unet_pred = self.unet(fuzzy_input)
        residual = sharp_target - unet_pred
        # Only use the coarse channel for VAE encoder
        encoder_input = torch.cat([unet_pred[:, 0:1], residual], dim=1)
        recon, mu, logvar = self.model(encoder_input, unet_pred[:, 0:1])
        recon_loss = F.l1_loss(recon, residual)
        kl_loss = kl_divergence(mu, logvar)
        total_loss = recon_loss + self.kl_weight * kl_loss
        return total_loss, recon_loss, kl_loss

    def training_step(self, batch, batch_idx):
        total_loss, recon_loss, kl_loss = self._loss(batch)
        self.log("train/loss", total_loss, on_step=True, on_epoch=True, prog_bar=True, sync_dist=True)
        self.log("train/recon_loss", recon_loss, on_step=True, on_epoch=True, sync_dist=True)
        self.log("train/kl_loss", kl_loss, on_step=True, on_epoch=True, sync_dist=True)
        return total_loss

    def validation_step(self, batch, batch_idx):
        total_loss, recon_loss, kl_loss = self._loss(batch)
        self.log("val/loss", total_loss, on_step=False, on_epoch=True, prog_bar=True, sync_dist=True)
        self.log("val/recon_loss", recon_loss, on_step=False, on_epoch=True, sync_dist=True)
        self.log("val/kl_loss", kl_loss, on_step=False, on_epoch=True, sync_dist=True)
        return total_loss

    def test_step(self, batch, batch_idx):
        total_loss, recon_loss, kl_loss = self._loss(batch)
        self.log("test/loss", total_loss, on_step=False, on_epoch=True, prog_bar=True, sync_dist=True)
        self.log("test/recon_loss", recon_loss, on_step=False, on_epoch=True, sync_dist=True)
        self.log("test/kl_loss", kl_loss, on_step=False, on_epoch=True, sync_dist=True)
        return total_loss

    def configure_optimizers(self):
        opt_cfg = self.hparams.get("optimizer", None)
        sch_cfg = self.hparams.get("scheduler", None)
        if opt_cfg and opt_cfg.get("type") == "AdamW":
            optimizer = torch.optim.AdamW(self.parameters(), lr=opt_cfg.get("lr", self.lr), betas=tuple(opt_cfg.get("betas", (0.5, 0.9))), weight_decay=opt_cfg.get("weight_decay", 1e-3))
        else:
            optimizer = torch.optim.Adam(self.parameters(), lr=opt_cfg.get("lr", self.lr) if opt_cfg else self.lr, weight_decay=opt_cfg.get("weight_decay", 1e-4) if opt_cfg else 1e-4)
        if sch_cfg and sch_cfg.get("type") == "ReduceLROnPlateau":
            scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=sch_cfg.get("patience", 3), factor=sch_cfg.get("factor", 0.25))
            return {"optimizer": optimizer, "lr_scheduler": {"scheduler": scheduler, "monitor": sch_cfg.get("monitor", "val/recon_loss"), "frequency": 1}}
        return optimizer

In [11]:
class LatentDenoiser(nn.Module):
    def __init__(self, in_channels, out_channels, channels, kernel_size):
        super().__init__()
        self.time_embed = nn.Sequential(
            nn.Linear(1, channels[0]),
            nn.ReLU(),
            nn.Linear(channels[0], channels[0])
        )


        self.unet = UNet(
            in_channels=in_channels + 2,  
            out_channels=out_channels,
            channels=channels,
            kernel_size=kernel_size
        )

    def forward(self, z_noisy, unet_pred, timestep=None):
        # z_noisy: [batch, channels, H, W]
        # unet_pred: [batch, 1, H, W]
        if timestep is None:
            timestep = torch.zeros(z_noisy.shape[0], 1, device=z_noisy.device)
        B, _, H, W = z_noisy.shape
        t_embed = self.time_embed(timestep.float())  # [batch, channels[0]]
        t_map = t_embed[:, :1].unsqueeze(-1).unsqueeze(-1).expand(-1, 1, H, W)
        x = torch.cat([z_noisy, unet_pred, t_map], dim=1)
        return self.unet(x)

In [12]:
class LDMLitModule(LightningModule):
    def __init__(self, vae, latent_dim=64, lr=1e-4, num_timesteps=50, noise_schedule="linear", loss_type="l2", hidden_dim=128, num_layers=4, optimizer=None, scheduler=None):
        super().__init__()
        self.save_hyperparameters(logger=False, ignore=['vae'])
        self.vae = vae.model.requires_grad_(False)
        self.vae.eval()
        self.denoiser = LatentDenoiser(1, 1, [hidden_dim]*num_layers, 3)
        self.lr = lr
        self.num_timesteps = num_timesteps
        self.loss_fn = {"l1": nn.L1Loss(), "l2": nn.MSELoss()}[loss_type]
        self.register_noise_schedule(noise_schedule)
        self.hparams.optimizer = optimizer
        self.hparams.scheduler = scheduler

    def register_noise_schedule(self, schedule="linear"):
        if schedule == "linear":
            betas = torch.linspace(1e-4, 2e-2, self.num_timesteps)
        elif schedule == "cosine":
            timesteps = torch.arange(self.num_timesteps + 1) / self.num_timesteps
            alphas = torch.cos(timesteps * torch.pi / 2) ** 2
            alphas = alphas / alphas[0]
            betas = 1 - alphas[1:] / alphas[:-1]
            betas = torch.clamp(betas, 0, 0.999)
        else:
            raise ValueError(f"Unknown schedule: {schedule}")

        #Setting up and storing noise schedules as buffers
        alphas = 1 - betas #beta here is the variance schedule (controls the noise added at each step)
        
        #How do we know how much signal remains after each timestep? 
        #This is essential because we need to separate signal from the noise added (fraction)
        alphas_cumprod = torch.cumprod(alphas, dim=0) #cumprod : cumulative product of elements along a given dimension.
        self.register_buffer('betas', betas)
        self.register_buffer('alphas_cumprod', alphas_cumprod)
        self.register_buffer('sqrt_alphas_cumprod', torch.sqrt(alphas_cumprod))
        self.register_buffer('sqrt_one_minus_alphas_cumprod', torch.sqrt(1 - alphas_cumprod))

    def q_sample(self, x_start, t, noise=None):
        if noise is None:
            noise = torch.randn_like(x_start)
        return (self.sqrt_alphas_cumprod[t].view(-1, 1) * x_start +
                self.sqrt_one_minus_alphas_cumprod[t].view(-1, 1) * noise)

    def p_losses(self, x_start, unet_pred, t, noise=None):
        if noise is None:
            noise = torch.randn_like(x_start)
        x_noisy = self.q_sample(x_start, t, noise)
        B, latent_dim = x_noisy.shape
        H = int(latent_dim ** 0.5)
        W = latent_dim // H
        x_noisy_4d = x_noisy.view(B, 1, H, W)
        # Downsample unet_pred to latent size
        unet_pred_ds = F.interpolate(unet_pred, size=(H, W), mode='bilinear', align_corners=False)
        predicted_noise = self.denoiser(x_noisy_4d, unet_pred_ds, t.unsqueeze(1).to(x_noisy.device))
        return self.loss_fn(predicted_noise, noise.view(B, 1, H, W))

    def forward(self, x, unet_pred):
        with torch.no_grad():
            mu, logvar = self.vae.encode(x)
            z = self.vae.reparameterize(mu, logvar)
        t = torch.randint(0, self.num_timesteps, (x.shape[0],), device=self.device)
        return self.p_losses(z, unet_pred, t)
    

    def training_step(self, batch, batch_idx):
        fuzzy_input, sharp_target = batch

        with torch.no_grad():
            unet_pred = self.vae.unet(fuzzy_input)
        residual = sharp_target - unet_pred
        encoder_input = torch.cat([unet_pred, residual], dim=1)
        loss = self.forward(encoder_input, unet_pred)
        self.log("train/loss", loss, on_step=True, on_epoch=True, prog_bar=True, sync_dist=True)
        return loss

    def validation_step(self, batch, batch_idx):
        loss = self.forward(batch[1])
        self.log("val/loss", loss, on_step=False, on_epoch=True, prog_bar=True, sync_dist=True)
        return loss

    def test_step(self, batch, batch_idx):
        loss = self.forward(batch[1])
        self.log("test/loss", loss, on_step=False, on_epoch=True, prog_bar=True, sync_dist=True)
        return loss

    @torch.no_grad()
    def sample(self, shape, unet_pred, num_steps=50):
        z = torch.randn(shape, device=self.device)
        H, W = ... # set to latent spatial size
        unet_pred_ds = F.interpolate(unet_pred, size=(H, W), mode='bilinear', align_corners=False)
        timesteps = torch.linspace(self.num_timesteps-1, 0, num_steps, dtype=torch.long, device=self.device)
        for i, t in enumerate(timesteps):
            t_batch = t.repeat(shape[0])
            predicted_noise = self.denoiser(z, unet_pred_ds, t_batch.unsqueeze(1))
            alpha_t = self.alphas_cumprod[t]
            alpha_t_prev = self.alphas_cumprod[t-1] if t > 0 else torch.tensor(1.0)
            pred_x0 = (z - torch.sqrt(1 - alpha_t) * predicted_noise) / torch.sqrt(alpha_t)
            if t > 0:
                noise = torch.randn_like(z) if i < len(timesteps) - 1 else 0
                z = torch.sqrt(alpha_t_prev) * pred_x0 + torch.sqrt(1 - alpha_t_prev) * noise
            else:
                z = pred_x0
        return z

    @torch.no_grad()
    def generate_samples(self, num_samples=1):
        z_samples = self.sample((num_samples, self.hparams.latent_dim))
        return self.vae.decode(z_samples)

    def configure_optimizers(self):
        opt_cfg = self.hparams.get("optimizer", None)
        sch_cfg = self.hparams.get("scheduler", None)
        if opt_cfg and opt_cfg.get("type") == "AdamW":
            optimizer = torch.optim.AdamW(self.parameters(), lr=opt_cfg.get("lr", self.lr), betas=tuple(opt_cfg.get("betas", (0.5, 0.9))), weight_decay=opt_cfg.get("weight_decay", 1e-3))
        else:
            optimizer = torch.optim.Adam(self.parameters(), lr=opt_cfg.get("lr", self.lr) if opt_cfg else self.lr, weight_decay=opt_cfg.get("weight_decay", 1e-4) if opt_cfg else 1e-4)
        if sch_cfg and sch_cfg.get("type") == "ReduceLROnPlateau":
            scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=sch_cfg.get("patience", 5), factor=sch_cfg.get("factor", 0.5))
            return {"optimizer": optimizer, "lr_scheduler": {"scheduler": scheduler, "monitor": sch_cfg.get("monitor", "val/loss"), "frequency": 1}}
        return optimizer

Training the hierarchy

In [13]:
def train_hierarchy(cfg):
    data_module = DownscalingDataModule(
        batch_size=cfg.data.batch_size,
        val_frac=cfg.data.val_split,
        test_frac=cfg.data.test_split,
        num_workers=cfg.data.num_workers,
        static_dir=cfg.paths.static_dir,
        save_stats_json=os.path.join(cfg.paths.output_dir, "stats.json")
    )
    data_module.setup()

    checkpoint_dir = cfg.paths.checkpoint_dir
    os.makedirs(checkpoint_dir, exist_ok=True)

    # Unet for mean pred
    unet_ckpt_dir = os.path.join(checkpoint_dir, "unet")
    os.makedirs(unet_ckpt_dir, exist_ok=True)
    unet_ckpts = sorted([f for f in os.listdir(unet_ckpt_dir) if f.endswith(".ckpt")])
    if unet_ckpts:
        print("Loading pretrained UNet ckpt")
        unet_module = UNetLitModule.load_from_checkpoint(os.path.join(unet_ckpt_dir, unet_ckpts[-1]))
    else:
        print("Training UNet")
        unet_module = UNetLitModule(
            net=UNet(
                in_channels=cfg.model.unet.in_channels,
                out_channels=cfg.model.unet.out_channels,
                channels=cfg.model.unet.channels,
                kernel_size=cfg.model.unet.kernel_size,
            ),
            lr=cfg.model.unet.lr,
            optimizer=cfg.optimizer.unet,
            scheduler=cfg.scheduler.unet,
        )
        unet_checkpoint = ModelCheckpoint(
            dirpath=unet_ckpt_dir,
            filename="unet-{epoch:02d}",
            save_top_k=1,
            monitor="val/loss",
            mode="min",
        )
        trainer_unet = Trainer(
            max_epochs=cfg.training.unet_epochs,
            accelerator=cfg.trainer.accelerator,
            gradient_clip_val=cfg.trainer.get("gradient_clip_val", 1.0),
            enable_checkpointing=True,
            logger=False,
            callbacks=[unet_checkpoint],
        )
        trainer_unet.fit(unet_module, datamodule=data_module)

    # Calculating residuals and also storing unet pred for conditoonal generation

    unet_preds = []
    residuals = []
    fuzzy_inputs = []

    for batch in data_module.train_dataloader():
        fuzzy_input, sharp_target = batch
        device = next(unet_module.parameters()).device
        fuzzy_input = fuzzy_input.to(device)
        sharp_target = sharp_target.to(device)

        with torch.no_grad():
            pred = unet_module.net(fuzzy_input)
            residual = sharp_target - pred
            # Save the full fuzzy_input for UNet, but only the coarse channel for VAE
            unet_pred_coarse = pred[:, 0:1]
            residual = residual.cpu()
            fuzzy_inputs.append(fuzzy_input)
            unet_preds.append(unet_pred_coarse)
            residuals.append(residual)
    fuzzy_inputs = torch.cat(fuzzy_inputs, dim=0)
    unet_preds = torch.cat(unet_preds, dim=0)
    residuals = torch.cat(residuals, dim=0)

# When creating ResidualsDataModule, use fuzzy_inputs (full), but in VAELitModule._loss, use only the coarse channel for VAE encoder:
# encoder_input = torch.cat([unet_pred[:, 0:1], residual], dim=1)

    #print(f"residual shape: {residuals.shape}")
    #print("residual nans:", torch.isnan(residuals).any()) 
    # Sometimes debugging step to check whether residuals are nans is required if loss becomes nan in VAE

    # VAE
    vae_ckpt_dir = os.path.join(checkpoint_dir, "vae")
    os.makedirs(vae_ckpt_dir, exist_ok=True)
    vae_ckpts = sorted([f for f in os.listdir(vae_ckpt_dir) if f.endswith(".ckpt")])
    if vae_ckpts:
        print("Loading pretrained VAE ckpt")
        vae_module = VAELitModule.load_from_checkpoint(os.path.join(vae_ckpt_dir, vae_ckpts[-1]))
    else:
        print("Training VAE")
        vae_module = VAELitModule(
            latent_dim=cfg.model.vae.latent_dim,
            lr=cfg.model.vae.lr,
            kl_weight=cfg.model.vae.kl_weight,
            input_channels=cfg.model.vae.input_channels,
            input_height=cfg.model.vae.input_height,
            input_width=cfg.model.vae.input_width,
            encoder_channels=cfg.model.vae.encoder_channels,
            decoder_channels=cfg.model.vae.decoder_channels,
            optimizer=cfg.optimizer.vae,
            scheduler=cfg.scheduler.vae,
            unet_module=unet_module
        )
        vae_checkpoint = ModelCheckpoint(
            dirpath=vae_ckpt_dir,
            filename="vae-{epoch:02d}",
            save_top_k=1,
            monitor="val/loss",
            mode="min",
        )
        residual_data_module = ResidualsDataModule(unet_preds, residuals, cfg.data.batch_size)
        residual_data_module.setup()


        trainer_vae = Trainer(
            max_epochs=cfg.training.vae_epochs,
            accelerator=cfg.trainer.accelerator,
            enable_checkpointing=True,
            logger=False,
            default_root_dir=vae_ckpt_dir,
            callbacks=[vae_checkpoint],
        )
        trainer_vae.fit(vae_module, datamodule=residual_data_module)



    # LDM
    ldm_module = LDMLitModule(
        vae=vae_module,
        latent_dim=cfg.model.ldm.latent_dim,
        lr=cfg.model.ldm.lr,
        num_timesteps=cfg.model.ldm.num_timesteps,
        noise_schedule=cfg.model.ldm.noise_schedule,
        loss_type=cfg.model.ldm.loss_type,
        hidden_dim=cfg.model.ldm.hidden_dim,
        num_layers=cfg.model.ldm.num_layers,
        optimizer=cfg.optimizer.ldm,
        scheduler=cfg.scheduler.ldm,
    )


    ldm_checkpoint = ModelCheckpoint(
        dirpath=os.path.join(checkpoint_dir, "ldm"),
        filename="ldm-{epoch:02d}",
        save_top_k=1,
        monitor="val/loss",
        mode="min",
    )

    residual_data_module = ResidualsDataModule(unet_preds, residuals, cfg.data.batch_size)
    residual_data_module.setup()
    trainer_ldm = Trainer(
        max_epochs=cfg.training.ldm_epochs,
        accelerator=cfg.trainer.accelerator,
        enable_checkpointing=True,
        logger=False,
        default_root_dir=os.path.join(checkpoint_dir, "ldm"),
        callbacks=[ldm_checkpoint],
    )
    trainer_ldm.fit(ldm_module, datamodule=residual_data_module)

Writing the YAML file

In [17]:
config = {
    "model": {
        "unet": {
            "in_channels": 19,
            "out_channels": 1,
            "lr": 1e-4,
            "channels": [32, 16],
            "kernel_size": 3
        },



        "vae": {
            "latent_dim": 32,
            "hidden_dim": 128,
            "input_channels": 2, #Input channels = predicted mean + residual
            "input_height": 672,
            "input_width": 576, #Dimensions of the input image
            "lr": 1e-4,
            "kl_weight": 0.001,
            "encoder_channels": [8],
            "decoder_channels": [8]
        },



        "ldm": {
            "latent_dim": 36,
            "lr": 1e-4,
            "num_timesteps": 20,
            "noise_schedule": "linear",
            "loss_type": "l2",
            "hidden_dim": 64,
            "num_layers": 2
        }



    },
    "trainer": {
        "max_epochs": 10,
        "min_delta": 1e-4,
        "accelerator": "gpu", # Changed from "auto" to "gpu"....make sure you run on a gpu, else it shall take ages
        "gradient_clip_val": 1.0
    },



    "training": {
        "unet_epochs": 20,
        "vae_epochs": 20,
        "ldm_epochs": 20
    },


    "optimizer": {
        "unet": {
            "type": "Adam",
            "lr": 1e-4,
            "weight_decay": 1e-4
        },



        "vae": {
            "type": "AdamW",
            "lr": 1e-4,
            "betas": [0.5, 0.9],
            "weight_decay": 1e-3
        },


        "ldm": {
            "type": "AdamW",
            "lr": 1e-4,
            "betas": [0.5, 0.9],
            "weight_decay": 1e-3
        }
    },
    "scheduler": {
        "unet": {
            "type": "ReduceLROnPlateau",
            "patience": 3,
            "factor": 0.25,
            "monitor": "val/loss"
        },
        "vae": {
            "type": "ReduceLROnPlateau",
            "patience": 3,
            "factor": 0.25,
            "monitor": "val/recon_loss"
        },
        "ldm": {
            "type": "ReduceLROnPlateau",
            "patience": 3,
            "factor": 0.25,
            "monitor": "val/loss"
        }
    },

    "data": {
        "batch_size": 32,
        "test_split": 0.15,
        "val_split": 0.15,
        "num_workers": 1
    },
    "paths": {
        "checkpoint_dir": "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/TA_work/checkpoints",
        "output_dir": "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/TA_work/outputs_dir",
        "data_dir": "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/TA_work/DiffScaler/data",
        "static_dir": "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/TA_work/DiffScaler/data/static_var"
    },
    "seed": 42
}

In [18]:
#Making dir
os.makedirs("conf", exist_ok=True)
with open("conf/config_experiments.yaml", "w") as f:
    yaml.dump(config, f)

In [16]:
cfg = OmegaConf.load("conf/config_experiments.yaml")
train_hierarchy(cfg)

Using 48 samples from 2020.
Using 48 samples from 2020.
Loading pretrained UNet ckpt


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.48 GiB. GPU 0 has a total capacity of 19.62 GiB of which 476.88 MiB is free. Including non-PyTorch memory, this process has 19.05 GiB memory in use. Of the allocated memory 12.55 GiB is allocated by PyTorch, and 6.29 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
def inference(cfg):
    # Denormalisation using stats from the json file
    stats_path = os.path.join(cfg.paths.output_dir, "stats.json")
    with open(stats_path, "r") as f:
        stats = json.load(f)
    low_2mt_mean = stats["low_2mt_mean"]
    low_2mt_std = stats["low_2mt_std"]

    def denormalise(arr):
        return arr * low_2mt_std + low_2mt_mean

    # Instantiate and set up the LightningDataModule
    data_module = DownscalingDataModule(
        batch_size=cfg.data.batch_size,
        val_frac=cfg.data.val_split,
        test_frac=cfg.data.test_split,
        num_workers=cfg.data.get("num_workers", 1),
        static_dir=cfg.paths.static_dir,
        save_stats_json=os.path.join(cfg.paths.output_dir, "stats.json")
    )
    data_module.setup()
    checkpoint_dir = cfg.paths.checkpoint_dir

    # Load ckpts for UNet, VAE, LDM
    unet_module = UNetLitModule.load_from_checkpoint(
        os.path.join(checkpoint_dir, "unet", sorted(os.listdir(checkpoint_dir + "/unet"))[-1])  #chosing the last ckpt
    )
    vae_module = VAELitModule.load_from_checkpoint(
        os.path.join(checkpoint_dir, "vae", sorted(os.listdir(checkpoint_dir + "/vae"))[-1])
    )
    ldm_module = LDMLitModule.load_from_checkpoint(
        os.path.join(checkpoint_dir, "ldm", sorted(os.listdir(checkpoint_dir + "/ldm"))[-1]),
        vae=vae_module
    )

    test_dataset = data_module.test_dataset

    # Eval() mode for inference
    unet_module.eval()
    vae_module.eval()
    ldm_module.eval()

    # visualising a single random test sample

    idx = random.randint(0, len(test_dataset)-1) #You can set the index (int) to compare
    sample = test_dataset[idx]
    if isinstance(sample, dict):
        fuzzy_input = torch.cat([
            F.interpolate(sample["low_2mT"].unsqueeze(0), size=sample["high_2mT"].shape[-2:], mode='bilinear', align_corners=False),
            sample["dem"].unsqueeze(0),
            sample["lat"].unsqueeze(0),
            sample["lc"].unsqueeze(0)
        ], dim=1).squeeze(0)

        sharp_target = sample["high_2mT"]
    else:
        fuzzy_input, sharp_target = sample

    with torch.no_grad():

        device = next(unet_module.parameters()).device
        fuzzy_input = fuzzy_input.unsqueeze(0).to(device)

        unet_pred = unet_module.net(fuzzy_input)

        input_height = vae_module.hparams.input_height // 2
        input_width = vae_module.hparams.input_width // 2
        latent_shape = (1, 1, input_height, input_width)

        # Downsample unet_pred to latent spatial size
        unet_pred_ds = F.interpolate(unet_pred, size=(input_height, input_width), mode='bilinear', align_corners=False)

        # Generate 3 samples
        generated_latents = []
        for _ in range(3):
            z_sample = ldm_module.sample(latent_shape, unet_pred_ds)
            z_sample_flat = z_sample.view(z_sample.size(0), -1)
            generated_latents.append(z_sample_flat)

        generated_latents = torch.cat(generated_latents, dim=0)

        # Decode latents to residuals
        generated_residuals = vae_module.model.decode(generated_latents, unet_pred)

        final_reconstructions = unet_pred + generated_residuals



    all_imgs = [
        denormalise(fuzzy_input[0, 0].cpu().numpy()),  # ERA5 predictor
        denormalise(final_reconstructions[0, 0].cpu().numpy()),        # Sample 1
        denormalise(final_reconstructions[1, 0].cpu().numpy()),        # Sample 2
        denormalise(final_reconstructions[2, 0].cpu().numpy()),        # Sample 3
        denormalise(sharp_target[0].cpu().numpy()),                    # Ground truth COSMO-CLM
    ]
    
    titles = [
        "ERA5 2m input (bilinearly interpolated to high res)",
        "Sample 1 of HR prediction",
        "Sample 2 of HR prediction",
        "Sample 3 of HR prediction",
        "Ground truth (COSMO-CLM)"
    ]
    vmin = min(img.min() for img in all_imgs)
    vmax = max(img.max() for img in all_imgs)


    # latlon extent for plotting

    lat = sample["lat"].cpu().numpy()[0]
    lon = sample["lon"].cpu().numpy()[0]
    lat_min, lat_max = lat.min(), lat.max()
    lon_min, lon_max = lon.min(), lon.max()
    extent = [lon_min, lon_max, lat_min, lat_max]

    fig, axes = plt.subplots(1, 5, figsize=(28, 6), constrained_layout=True)
    images = []

    for i, ax in enumerate(axes):
        im = ax.imshow(all_imgs[i], cmap='coolwarm', vmin=vmin, vmax=vmax, extent=extent, origin='lower')
        ax.set_title(titles[i], fontsize=15, fontweight='bold')
        ax.set_xlabel("Longitude", fontsize=13)
        ax.set_ylabel("Latitude", fontsize=13)
        ax.tick_params(axis='both', which='major', labelsize=11)
        ax.grid(False, which='both')
        images.append(im)

    cbar = fig.colorbar(images[0], ax=axes, orientation='horizontal', fraction=0.04, pad=0.08)
    cbar.set_label("2m Temperature (degrees C)", fontsize=15, fontweight='bold')
    cbar.ax.tick_params(labelsize=13)

    plt.suptitle("ERA5 low res + 3 LDM Samples + COSMO-CLM Ground Truth", fontsize=20, y=1.05, fontweight='bold')


    plt.show()



In [ ]:
#Now that the models are trained,,, how will you perform inference using the trained hierarchy?
inference(cfg)

Using 48 samples from 2020.
Using 48 samples from 2020.


KeyError: 'lon'